In [1]:
import pandas as pd
import os,sys,re
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.chdir("/datadrive/data_adivse")

In [3]:
os.listdir()

['sentiments_list.txt',
 'test_images',
 'train',
 'test-set.zip',
 'sample_response.txt',
 'train_images',
 'test',
 'topics_list.txt',
 'OUTPUT_FILE_INSTRUCTIONS.txt',
 'README.txt']

In [4]:
img=cv2.imread("train_images/0/100040.jpg")

In [30]:
from keras.preprocessing import image

img_initial = image.load_img("train_images/0/100040.jpg")
img = img_initial.resize((128,128))

img_array = image.img_to_array(img)
#img_array = np.expand_dims(img_array, axis = 0)

In [31]:
img_array.shape

(128, 128, 3)

In [7]:
os.listdir()

['sentiments_list.txt',
 'test_images',
 'train',
 'test-set.zip',
 'sample_response.txt',
 'train_images',
 'test',
 'topics_list.txt',
 'OUTPUT_FILE_INSTRUCTIONS.txt',
 'README.txt']

In [8]:
os.listdir('train')

['QA_Combined_Action_Reason_train.json',
 'Symbols_train.json',
 'Topics_train.json',
 'Slogans_train.json',
 'Strategies_train.json',
 'Sentiments_train.json']

In [9]:
g='train/Topics_train.json'
key="topics"
with open(g,encoding ='utf-8') as f:
    json_dict={}
    json_dict[key]=json.load(f)


In [10]:
k="0/100040.jpg"

In [11]:
for l in json_dict['topics'].keys():
    if l == k:
        print(json_dict['topics'][l])

['3', '3', '3', '3', '3']


In [12]:

g=[]
u= 0
for key in json_dict['topics'].keys():
    #df=pd.DataFrame(index=range(1),columns=['image','topics'])
    df={}
    df['image'] = str(key)
    #print(str(key))
    #print(len(json_dict['topics'][key]))
    #print(json_dict['topics'][key])
    df['topics']=json_dict['topics'][key]
    #df['topics']=','.join(json_dict['topics'][key])

    #print(df)
    g.append(df)
    
    u += 1
    #if u >= 100:
        #break

In [13]:
df

{'image': '9/21309.jpg', 'topics': ['28', '28', '28']}

In [14]:
g[3]

{'image': '10/173485.png', 'topics': ['8', '8', '8']}

In [15]:
g2=pd.DataFrame(g)

In [16]:
def cleaner(d):
    d=re.sub('[^0-9]','',d)
    return d
def classcleaner(x):
    return [cleaner(h) for h in x if len(cleaner(h)) > 0 ]

In [17]:
g2['cleantopics']=g2['topics'].apply(lambda x : classcleaner(x))

In [18]:
g2.head()

,image,topics,cleantopics
0,7/90807.jpg,"[12, 38, 12]","[12, 38, 12]"
1,9/154989.jpg,"[23, 23, 23]","[23, 23, 23]"
2,7/153887.jpg,"[6, 6, 6]","[6, 6, 6]"
3,10/173485.png,"[8, 8, 8]","[8, 8, 8]"
4,5/10075.jpg,"[6, 6, 6]","[6, 6, 6]"


In [19]:
# from sklearn.preprocessing import MultiLabelBinarizer
# one_hot = MultiLabelBinarizer()
onehe = g2['cleantopics'].str.join('|').str.get_dummies()


In [20]:
labels=pd.concat([g2[['image']],onehe],axis=1)

In [21]:
labels.index=labels['image']
del labels['image']

In [22]:
labels.head()

,1,10,11,12,13,14,15,16,17,18,...,36,37,38,39,4,5,6,7,8,9
image,,,,,,,,,,,,,,,,,,,,,
7/90807.jpg,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9/154989.jpg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7/153887.jpg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10/173485.png,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5/10075.jpg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [23]:
labels.loc['7/90807.jpg'].values

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [35]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

def cnnmodel(nclass):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(128,128,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nclass))
    model.add(Activation('sigmoid'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model



In [36]:
## creating partition dictionary for train and validation
train_data_dir = 'train_images/0/'
traindataids=os.listdir(train_data_dir)
import random 
random.shuffle(traindataids)
partition={}
train_ids=random.sample(traindataids,int(.9*len(traindataids)))
val_ids= [id for id in traindataids if id not in train_ids ]



In [37]:
len(val_ids),len(train_ids)

(463, 4164)

In [38]:
partition['train'] =train_ids
partition['validation'] =val_ids

In [39]:
list_IDs=traindataids

In [51]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels,train_data_dir, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=39, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.train_data_dir=train_data_dir
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size,self.n_classes), dtype=int)
        
        


        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #img=cv2.imread(train_data_dir + ID)
            img_initial = image.load_img(self.train_data_dir + ID)
            img = img_initial.resize((128,128))

            img_array = image.img_to_array(img)
            
            
            X[i,] = img_array

            # Store class
            y[i] = self.labels.loc['0/'+ID].values #self.labels[ID]  #y[i]

        return X, y
    #return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [52]:
import numpy as np
train_data_dir = 'train_images/0/'
from keras.models import Sequential
# from my_classes import DataGenerator

# Parameters
params = {'dim': (128,128),
          'batch_size': 32,
          'n_classes': 39,
          'n_channels': 3,
          'shuffle': True}

# Datasets
partition = partition
labels = labels

# Generators
training_generator = DataGenerator(partition['train'], labels,train_data_dir, **params)
validation_generator = DataGenerator(partition['validation'], labels,train_data_dir, **params)



In [53]:
# Design model
cnnm=cnnmodel(39)



In [54]:
cnnm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 61, 61, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 64)        18496     
__________

In [ ]:
# Train model on dataset
cnnm.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                   use_multiprocessing=True,epochs = 200,
                    workers=3)

Epoch 1/200
130/130 [==============================] - 16s 124ms/step - loss: 13.8080 - acc: 0.0800 - val_loss: 16.1242 - val_acc: 0.0938
Epoch 1/200
Epoch 2/200
130/130 [==============================] - 25s 191ms/step - loss: 16.5429 - acc: 0.0868 - val_loss: 16.5958 - val_acc: 0.0982
Epoch 3/200
Epoch 2/200
130/130 [==============================] - 18s 140ms/step - loss: 15.8832 - acc: 0.0988 - val_loss: 17.1742 - val_acc: 0.0960
Epoch 4/200
130/130 [==============================] - 15s 118ms/step - loss: 15.4321 - acc: 0.0990 - val_loss: 17.0668 - val_acc: 0.0982
Epoch 5/200
130/130 [==============================] - 16s 121ms/step - loss: 15.3903 - acc: 0.1031 - val_loss: 17.3979 - val_acc: 0.0960
Epoch 6/200
 11/130 [=>............................] - ETA: 12s - loss: 15.7776 - acc: 0.0994

In [50]:
32*3 + 32

128